# **Item 3 - Sobre GenAI e LLMs**

## Problema

<p align="justify"> O Dataset disponibilizado tem 2 colunas de texto, uma de título e outra de descrição. Utilize um LLM - como  ChatGPT ou Claude - para criar features dos produtos que serão úteis para análise. </p>


<a name='1'></a>

## 1 - Carregando dados e modelo

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, GenerationConfig, TrainingArguments, Trainer, AutoTokenizer, AutoModel
import torch
import os
import time
import evaluate
import pandas as pd
import numpy as np

In [1]:
os.chdir("..")


<a name='1.1'></a>
### 1.2 - Dataset e LLM loading

Carregando os dados localmente.

In [ ]:
import os
import pyarrow as pa

# Defina o caminho para a pasta onde os arquivos Arrow estão localizados
folder_path = "artifacts/data_ingestion"

# Liste os arquivos na pasta
file_list = [f for f in os.listdir(folder_path) if f.endswith(".arrow")]

# Leia cada arquivo Arrow
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    table = pa.ipc.RecordBatchFileReader(file_path).read_all()
    
    # Faça o que você precisa com a tabela, por exemplo, converta-a para um DataFrame
    df = table.to_pandas()
    
    # Agora você pode trabalhar com o DataFrame df

    print(f"Leu o arquivo: {file_name}")


In [3]:
dataset = load_dataset("spacemanidol/product-search-corpus", 
                       split="train")
dataset

Dataset({
    features: ['docid', 'title', 'text'],
    num_rows: 1118658
})

In [33]:
tokenizer = AutoTokenizer.from_pretrained("spacemanidol/trec-product-search-all-miniLM-L6-v2")
model = AutoModel.from_pretrained("spacemanidol/trec-product-search-all-miniLM-L6-v2")

<a name='1.2'></a>
### 1.2 - Testando o modelo com Zero Shot Inferencing


Testando o modelo com zero shot inferencing, podemos ver que o modelo se esforça para resumir o diálogo em comparação com o resumo da linha de base, e extrai algumas informações importantes do texto, o que indica que o modelo pode ser ajustado para a tarefa em questão.

In [51]:
index = 2

In [52]:
title = dataset[index]["title"]

In [36]:
category = "Phone Accessories"

In [53]:
# Montar o prompt de classificação de texto
prompt = f"Classify the following product title into its category:\n\n{title}\n\nCandidate categories: Supplies, Phone Accessories, Clothing, Book, Electronics, Shoes, and others."

# prompt = f"Do whatever {title}"

In [54]:
# Tokenizar o prompt e codificá-lo para uso com o modelo
inputs = tokenizer(prompt, return_tensors="pt")

In [42]:
# # Executar a classificação de texto
# with torch.no_grad():
#     outputs = model(**inputs)
#     logits = outputs.logits

# # Obter a classe prevista
# predicted_class = torch.argmax(logits, dim=1).item()

In [55]:
# Obter o pooler_output após a passagem pelo modelo
with torch.no_grad():
    model_outputs = model(**inputs)
    pooler_output = model_outputs.pooler_output

In [56]:
#  Adicionar uma camada de classificação personalizada
class CustomClassifier(torch.nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()
        self.num_classes = 5  # Defina o número correto de classes aqui
        self.classifier = torch.nn.Linear(model.config.hidden_size, self.num_classes)

    def forward(self, pooled_output):
        return self.classifier(pooled_output)

In [57]:
classifier = CustomClassifier()
outputs = classifier(pooler_output)

# Obter a classe prevista
predicted_class = torch.argmax(outputs, dim=1).item()

In [58]:
predicted_class

4

## O dataset é extraído da Amazon, por isso agrupei essas categorias.

In [107]:
tokenizer = AutoTokenizer.from_pretrained("spacemanidol/trec-product-search-bge-base-en")
model = AutoModel.from_pretrained("spacemanidol/trec-product-search-bge-base-en")

In [108]:
if torch.cuda.is_available():
    # Mova o modelo para a GPU
    model.to('cuda')
    
    # Criar uma lista para armazenar as previsões
    predictions = []
    
    categories = {
                0: "Food",
                1: "Health, Beauty",
                2: "Electronics, Computers",
                3: "Home and Kitchen",
                4: "Clothing",
                5: "Sports",
                6: "Office",
                7: "Movies, Music",
                8: "Books"     
            }
    # Iterar sobre todos os exemplos no seu conjunto de dados
    for i, example in enumerate(dataset):
        title = example["title"]

        # Montar o prompt de classificação de texto
        prompt = f"Classify the following product title into its category:\n\n{title}\n\nCandidate categories: {list(categories.values())}."

        # Tokenizar o prompt e codificá-lo para uso com o modelo
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Mova os inputs para a GPU
        
        # Obter o pooler_output após a passagem pelo modelo
        with torch.no_grad():
            model_outputs = model(**inputs)
            pooler_output = model_outputs.pooler_output
        
        # Adicionar uma camada de classificação personalizada
        class CustomClassifier(torch.nn.Module):
            def __init__(self):
                super(CustomClassifier, self).__init__()
                self.num_classes = 5  # Defina o número correto de classes aqui
                self.classifier = torch.nn.Linear(model.config.hidden_size, self.num_classes).to('cuda')  # Mova o classificador para a GPU

            def forward(self, pooled_output):
                return self.classifier(pooled_output)
        
        classifier = CustomClassifier()
        outputs = classifier(pooler_output)
        
        # Obter a classe prevista
        predicted_class = torch.argmax(outputs, dim=1).item()
        
        # Armazenar a previsão na lista de previsões
        predictions.append(predicted_class)
        predicted_categories = [categories[index] for index in predictions]
        
        # Imprimir o número atual do exemplo processado
        print(f"Exemplo {i + 1}/{len(dataset)} processado")

else:
    print("GPU não disponível. Usando CPU.")

Exemplo 1/1118658 processado
Exemplo 2/1118658 processado
Exemplo 3/1118658 processado
Exemplo 4/1118658 processado
Exemplo 5/1118658 processado
Exemplo 6/1118658 processado
Exemplo 7/1118658 processado
Exemplo 8/1118658 processado
Exemplo 9/1118658 processado
Exemplo 10/1118658 processado
Exemplo 11/1118658 processado
Exemplo 12/1118658 processado
Exemplo 13/1118658 processado
Exemplo 14/1118658 processado
Exemplo 15/1118658 processado
Exemplo 16/1118658 processado
Exemplo 17/1118658 processado
Exemplo 18/1118658 processado
Exemplo 19/1118658 processado
Exemplo 20/1118658 processado
Exemplo 21/1118658 processado
Exemplo 22/1118658 processado
Exemplo 23/1118658 processado
Exemplo 24/1118658 processado
Exemplo 25/1118658 processado
Exemplo 26/1118658 processado
Exemplo 27/1118658 processado
Exemplo 28/1118658 processado
Exemplo 29/1118658 processado
Exemplo 30/1118658 processado
Exemplo 31/1118658 processado
Exemplo 32/1118658 processado
Exemplo 33/1118658 processado
Exemplo 34/1118658 

KeyboardInterrupt: 

In [110]:
predicted_categories

['Clothing',
 'Home and Kitchen',
 'Electronics, Computers',
 'Food',
 'Clothing',
 'Clothing',
 'Electronics, Computers',
 'Electronics, Computers',
 'Home and Kitchen',
 'Electronics, Computers',
 'Clothing',
 'Clothing',
 'Clothing',
 'Electronics, Computers',
 'Health, Beauty',
 'Food',
 'Health, Beauty',
 'Clothing',
 'Electronics, Computers',
 'Home and Kitchen',
 'Health, Beauty',
 'Home and Kitchen',
 'Food',
 'Food',
 'Electronics, Computers',
 'Home and Kitchen',
 'Food',
 'Clothing',
 'Electronics, Computers',
 'Clothing',
 'Clothing',
 'Clothing',
 'Clothing',
 'Food',
 'Clothing',
 'Home and Kitchen',
 'Clothing',
 'Health, Beauty',
 'Clothing',
 'Clothing',
 'Health, Beauty',
 'Food',
 'Health, Beauty',
 'Clothing',
 'Food',
 'Health, Beauty',
 'Food',
 'Home and Kitchen',
 'Home and Kitchen',
 'Health, Beauty',
 'Clothing',
 'Food',
 'Clothing',
 'Food',
 'Home and Kitchen',
 'Food',
 'Home and Kitchen',
 'Health, Beauty',
 'Electronics, Computers',
 'Health, Beauty',
 '

In [ ]:
# # Verifique se uma GPU está disponível
# if torch.cuda.is_available():
#     # Mova o modelo para a GPU
#     model.to('cuda')
    
#     # Criar uma lista para armazenar as previsões
#     predictions = []
#     categories = {
#                 0: "Food",
#                 1: "Automotive",
#                 2: "Health, Beauty",
#                 3: "Electronics, Computers",
#                 4: "Arts, Crafts",
#                 5: "Home and Kitchen",
#                 6: "Clothing and Accessories",
#                 7: "Sports",
#                 8: "Office",
#                 9: "Movies, Music",
#                 10: "Books"     
#             }
    
#     # Iterar sobre todos os exemplos no seu conjunto de dados
#     for example in dataset:
#         title = example["title"]

#         # Montar o prompt de classificação de texto
#         prompt = f"Classify the following product title into its category:\n\n{title}\n\nCandidate categories: {list(categories.values())}."

#         # Tokenizar o prompt e codificá-lo para uso com o modelo
#         inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Mova os inputs para a GPU
        
#         # Obter o pooler_output após a passagem pelo modelo
#         with torch.no_grad():
#             model_outputs = model(**inputs)
#             pooler_output = model_outputs.pooler_output

#         # Adicionar uma camada de classificação personalizada
#         class CustomClassifier(torch.nn.Module):
#             def __init__(self):
#                 super(CustomClassifier, self).__init__()
#                 self.num_classes = len(categories)  # Defina o número correto de classes aqui
#                 self.classifier = torch.nn.Linear(model.config.hidden_size, self.num_classes).to('cuda')  # Mova o classificador para a GPU

#             def forward(self, pooled_output):
#                 return self.classifier(pooled_output)
        
#         classifier = CustomClassifier()
#         outputs = classifier(pooler_output)

#         # Obter a classe prevista
#         predicted_class = torch.argmax(outputs, dim=1).item()

#         # Armazenar a previsão na lista de previsões
#         predictions.append(predicted_class)
#         predicted_categories = [categories[index] for index in predictions]
#         print(f"Exemplo {i + 1}/{len(dataset)} processado")
# else:
#     print("GPU não disponível. Usando CPU.")


In [97]:
predicted_categories

['Automotive',
 'Automotive',
 'Automotive',
 'Health, Beauty',
 'Health, Beauty',
 'Food',
 'Arts, Crafts',
 'Health, Beauty',
 'Arts, Crafts',
 'Food',
 'Electronics, Computers',
 'Automotive',
 'Electronics, Computers',
 'Automotive',
 'Electronics, Computers',
 'Arts, Crafts',
 'Health, Beauty',
 'Electronics, Computers',
 'Electronics, Computers',
 'Automotive',
 'Electronics, Computers',
 'Automotive',
 'Electronics, Computers',
 'Arts, Crafts',
 'Arts, Crafts',
 'Arts, Crafts',
 'Arts, Crafts',
 'Automotive',
 'Electronics, Computers',
 'Arts, Crafts',
 'Electronics, Computers',
 'Electronics, Computers',
 'Arts, Crafts',
 'Food',
 'Electronics, Computers',
 'Electronics, Computers',
 'Electronics, Computers',
 'Arts, Crafts',
 'Arts, Crafts',
 'Automotive',
 'Automotive',
 'Health, Beauty',
 'Food',
 'Health, Beauty',
 'Food',
 'Automotive',
 'Automotive',
 'Food',
 'Health, Beauty',
 'Food',
 'Arts, Crafts',
 'Arts, Crafts',
 'Automotive',
 'Food',
 'Food',
 'Electronics, Comp

In [104]:
len(predicted_categories)

549984

In [102]:
df_original = dataset.to_pandas().head()

,docid,title,text
0,1,FYY Leather Case with Mirror for Samsung Galax...,Product Description Premium PU Leather Top qua...
1,2,"Playtex Women's 18 Hour Easy On, Easy Off Fron...",Product Description Introducing Playtex 18 hou...
2,4,YUEPIN U-Tube Clamp 304 Stainless Steel Hose P...,Product Description Specification: Material: 3...
3,5,Bruce's Big Storm (Mother Bruce Series),
4,6,DJI Shoulder Neck Strap Belt Sling Lanyard Nec...,Product Description Specifications: Item Condi...
...,...,...,...
1118653,1661902,SMARTSTANDARD 10ft Heavy Duty Double Door Slid...,From the brand Previous page SmartStandard is ...
1118654,1661903,Men's DMC V Cosplay Costume Full Set Outfit,Product Description Size Chart: X-Small: Heigh...
1118655,1661905,OYATON Christmas Votive Candle Holders with Sn...,From the brand Christmas holiday decorations F...
1118656,1661906,BirdRock Home 5 Gallon Stainless Steel Beverag...,From the brand Previous page Holiday Home Deco...


In [98]:
# Criar um DataFrame a partir da lista de previsões
data = {'Predicted_Category': predicted_categories}
df = pd.DataFrame(data)

In [106]:
df['Predicted_Category'].value_counts()

Predicted_Category
Automotive                110392
Arts, Crafts              110202
Food                      109888
Electronics, Computers    109772
Health, Beauty            109730
Name: count, dtype: int64

In [100]:
# Salvar o DataFrame em um arquivo CSV
df.to_parquet('categories.parquet', index=False, compression="gzip")

In [ ]:
# Criar um DataFrame a partir da lista de previsões
data = {'Predicted_Category': predicted_categories}
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_parquet('categories.parquet', index=False, compression="gzip")

predicted_category

In [7]:
dataset_prompt_eng = dataset.select(range(50))

In [9]:
for i in range(50):
    print(dataset_prompt_eng[i]["title"])
    print()

FYY Leather Case with Mirror for Samsung Galaxy S8 Plus, Leather Wallet Flip Folio Case with Mirror and Wrist Strap for Samsung Galaxy S8 Plus Black

Playtex Women's 18 Hour Easy On, Easy Off Front & Back Close Post Surgery Bra US400C

YUEPIN U-Tube Clamp 304 Stainless Steel Hose Pipe Cable Strap Clips With Rubber Cushioned (1-21/32"(42mm)-10pcs)

Bruce's Big Storm (Mother Bruce Series)

DJI Shoulder Neck Strap Belt Sling Lanyard Necklaces for Dji Phantom 3 Inspire 1 Remote

Crocs Jibbitz 5-Pack Alien Shoe Charms | Jibbitz for Crocs

Design Toscano QM2787100 Darby, the Forest Fawn Baby Deer Statue,full color

Zeagoo Women's Open Front Cardigan 3/4 Sleeve Draped Ruffles Soft Knit Sweaters

Rekayla Open Toe Tie Up Ankle Wrap Flat Sandals for Women

Apron for Women, Waterproof Adjustable Bib Cooking Aprons with Pocket-2 Side Coral Velvet Towels Stitched Durable Pinstripe Aprons for Dishwashing, Baking, Grill, Restaurant Even Garden Craft

DIY 5D Diamond Painting by Number Kit for Adult, F

In [ ]:
["Phone Accessories", "Clothing", " Supplies", "Book", "Clothing", "Shoes", "Ornaments", "Clothing",
 "Shoes", "Clothing", "Shoes"]

In [24]:
dataset_prompt_eng.add_column()

Dataset({
    features: ['docid', 'title', 'text'],
    num_rows: 50
})

In [15]:
type(dataset)

datasets.arrow_dataset.Dataset

<a name='2'></a>
## 2 - Aplicando Full Fine-Tuning

<a name='2.1'></a>
### 2.1 - Preprocessando o Dialog-Summary Dataset

Precisamos converter os pares de resumo de diálogo (prompt-response) em instruções explícitas para o LLM. Vamos anexar uma instrução ao início da caixa de diálogo com `Resumir a seguinte conversa` e ao início do resumo com `Resumo` da seguinte forma:

Training prompt (dialogue):
```
Summarize the following conversation.

    Chris: This is his part of the conversation.
    Antje: This is her part of the conversation.
    
Summary:
```

Training response (summary):
```
Both Chris and Antje participated in the conversation.
```

Em seguida, vamos preprocessar o conjunto de dados de resposta de prompt em tokens e extrair seus `input_ids` (1 por token).

In [7]:
def tokenize_function(example):
    start_prompt = 'Extract the product category from the product title.\n\n'
    end_prompt = '\n\nCategory: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# Na verdade, o conjunto de dados contém 3 divisões: treino, validação, teste.
# O código tokenize_function está manipulando todos os dados em todas as divisões em lotes.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Loading cached processed dataset at /home/bruno/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9a0a8356ed329662.arrow
Loading cached processed dataset at /home/bruno/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-4a7759ff139dac8d.arrow
Loading cached processed dataset at /home/bruno/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-44843119538026ee.arrow


Para poupar tempo, vamos pegar um sample do conjunto de dados:

In [8]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Loading cached processed dataset at /home/bruno/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-5fe7fa0f459c48fe.arrow
Loading cached processed dataset at /home/bruno/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-4471a715ac15ad6c.arrow
Loading cached processed dataset at /home/bruno/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-f3a440617082d3f8.arrow


Verifica as formas de todas as três partes do conjunto de dados:



In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


<a name='2.2'></a>
### 2.2 - Fine-Tune do modelo com o conjunto de dados pré-processado

Vamos utilizar a classe interna Hugging Face `Trainer` (consulte a documentação [aqui](https://huggingface.co/docs/transformers/main_classes/trainer)). Passaremos o conjunto de dados pré-processado com referência ao modelo original. Outros parâmetros de treinamento são encontrados experimentalmente e não há necessidade de entrar em detalhes sobre eles no momento.

In [10]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [13]:
tokenized_datasets['validation']

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 5
})

In [12]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 125
})

In [10]:
#trainer.train()

Treinar uma versão "fully fine-tuned" do modelo levaria algumas horas em uma GPU. Para economizar tempo, estarei somente lendo um ponto de checkpoint do modelo fully fine-tuned para usar no restante deste notebook. Esse modelo será chamado de **instruct model**.

Crie uma instância da classe `AutoModelForSeq2SeqLM` para o instruct model:

In [11]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)

<a name='2.3'></a>
### 2.3 - Avaliação do modelo qualitativamente (avaliação humana)

Como em muitos aplicativos GenAI, uma abordagem qualitativa em que você se pergunta: "Meu modelo está se comportando da maneira que deveria?" geralmente é um bom ponto de partida. No exemplo abaixo (o mesmo com o qual iniciamos esse notebook), podemos ver como o instruct model é capaz de criar um resumo razoável do diálogo em comparação com o original, que foi incapaz de entender o que está sendo solicitado.

In [14]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

<a name='2.4'></a>
### 2.4 - Avaliação do modelo quantitativamente (com a métrica ROUGE)

A [métrica ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) ajuda a quantificar a validade dos resumos produzidos por modelos. Ele compara resumos a um resumo de "linha de base" que geralmente é criado por um ser humano. Embora não seja perfeito, indica o aumento geral na eficácia do resumo que conseguimos com o fine tune.

In [ ]:
rouge = evaluate.load('rouge')

Vamos gerar as saídas para a amostra do conjunto de dados de teste (apenas 10 diálogos e resumos para economizar tempo) e salvar os resultados.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: I need to take a dictation for you.,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,#Person1#: I need to take a dictation for you.,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1#: I need to take a dictation for you.,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Happy birthday, Brian. #Person2#: I...",Brian's birthday is coming. #Person1# invites ...


Avaliando os modelos computando as métricas ROUGE, observe a melhora nos resultados!

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}
INSTRUCT MODEL:
{'rouge1': 0.3839073578147069, 'rouge2': 0.1605940055118149, 'rougeL': 0.285893894130932, 'rougeLsum': 0.2876203213812496}


O arquivo `data/dialogue-summary-training-results.csv` contém uma lista pré-preenchida de todos os resultados do modelo que você pode usar para avaliar em uma seção maior de dados. Vamos fazer isso para cada um dos modelos:

In [ ]:
results = pd.read_csv("data/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}


Os resultados mostram uma melhoria substancial em todas as métricas ROUGE:

In [ ]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 18.82%
rouge2: 10.43%
rougeL: 13.70%
rougeLsum: 13.69%


<a name='3'></a>
## 3 - Parameter Efficient Fine-Tuning (PEFT)

Agora, vamos executar o **(PEFT)** em oposição ao "fully fine tune" como fizemos acima. O PEFT é uma forma de fine tune de instrução, que é muito mais eficiente do que o fully fine tune - com resultados de avaliação comparáveis, como você verá em breve.

PEFT é um termo genérico que inclui o **Low-Rank Adaptation (LoRA)** e ajuste de prompt (que NÃO É O MESMO que engenharia de prompt!). Na maioria dos casos, quando alguém diz PEFT, geralmente significa LoRA. O LoRA, em um nível muito alto, permite ao usuário ajustar seu modelo usando menos recursos de computação (em alguns casos, uma única GPU). Após o fine tue para uma tarefa ou caso de uso com LoRA, o resultado é que o LLM original permanece inalterado e surge um “adaptador LoRA” recém-treinado. Este adaptador LoRA é muito, muito menor que o LLM original - da ordem de um único dígito % do tamanho original do LLM (MBs vs GBs).

Dito isso, no momento da inferência, o adaptador LoRA precisa ser reunido e combinado com seu LLM original para atender à solicitação de inferência. O benefício, no entanto, é que muitos adaptadores LoRA podem reutilizar o LLM original, o que reduz os requisitos gerais de memória ao atender a várias tarefas e casos de uso.

<a name='3.1'></a>
### 3.1 - Configurando o modelo PEFT/LoRA para Fine-Tuning

Precisamos configurar o modelo PEFT/LoRA para fine tune com um novo adaptador de camada/parâmetro. Usando PEFT/LoRA, estamos congelando a LLM subjacente e apenas treinando o adaptador. Dê uma olhada na configuração do LoRA abaixo. Observe o hiperparâmetro rank (`r`), que define o rank/dimensão do adaptador a ser treinado.

In [20]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Adicione camadas/parâmetros do adaptador LoRA ao LLM original a ser treinado.

In [21]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


<a name='3.2'></a>
### 3.2 - Treinando o adaptador PEFT

Vamos definir os argumentos de treinamento e criar a instância `Trainer`.

In [22]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

NameError: name 'tokenized_datasets' is not defined

In [ ]:
#peft_trainer.train()

#peft_model_path="./peft-dialogue-summary-checkpoint-local"

#peft_trainer.model.save_pretrained(peft_model_path)
#tokenizer.save_pretrained(peft_model_path)

('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

Em seguida, vamos carregar uma versão com o Lora já aplicado. Perceba a diferença no tamanho do arquivo!

Vamos agora preparar o modelo adicionando um adaptador ao FLAN-T5 original. Você está configurando `is_trainable=False` porque o plano é apenas realizar inferência com este modelo PEFT. Se você estivesse preparando o modelo para treinamento adicional, definiria `is_trainable=True`.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-from-s3/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

O número de parâmetros treináveis ​​será `0` devido à configuração `is_trainable=False`:

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


<a name='3.3'></a>
### 3.3 - Avalie o modelo qualitativamente (avaliação humana)

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: I'm thinking of upgrading my computer.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# suggests #Person2# adding a painting program to #Person2#'s software and upgrading the hardware. #Person2# also wants to add a CD-ROM drive.
---------------------------------------------------------------------------------------------------
PEFT MODEL: #Person1# recommends adding a painting program to #Person2#'s software and upgrading hardware. #Person2# also wants to upgrade the hardware because it's outdated now.


<a name='3.4'></a>
### 3.4 - Avalia os modelos quantitativamente (com a métrica ROUGE)


In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: I need to take a dictation for you.,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,#Person1#: I need to take a dictation for you.,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1#: I need to take a dictation for you.,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
5,#Person2# complains to #Person1# about the tra...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Happy birthday, Brian. #Person2#: I...",Brian's birthday is coming. #Person1# invites ...,Brian remembers his birthday and invites #Pers...


In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.24089921652421653, 'rouge2': 0.11769053708439897, 'rougeL': 0.22001958689458687, 'rougeLsum': 0.22134175465057818}
INSTRUCT MODEL:
{'rouge1': 0.3839073578147069, 'rouge2': 0.1605940055118149, 'rougeL': 0.285893894130932, 'rougeLsum': 0.2876203213812496}
PEFT MODEL:
{'rouge1': 0.3725351062275605, 'rouge2': 0.12138811933618107, 'rougeL': 0.27620639623170606, 'rougeLsum': 0.2758134870822362}


Observe que os resultados do modelo PEFT não são tão ruins, enquanto o processo de treinamento foi muito mais fácil!

Nós já calculamos a pontuação ROUGE no conjunto de dados completo, depois de carregar os resultados do arquivo `data/dialogue-summary-training-results.csv`. Vamso carregar agora os valores para o modelo PEFT e verificar seu desempenho em comparação com outros modelos.

In [ ]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}
PEFT MODEL:
{'rouge1': 0.40810631575616746, 'rouge2': 0.1633255794568712, 'rougeL': 0.32507074586565354, 'rougeLsum': 0.3248950182867091}


Os resultados mostram uma melhoria menor em relação ao fully fine tune, mas os benefícios do PEFT geralmente superam as métricas de desempenho ligeiramente inferiores.

Calcule a melhoria do PEFT em relação ao modelo original:

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 17.47%
rouge2: 8.73%
rougeL: 12.36%
rougeLsum: 12.34%


Agora calcule a melhoria do PEFT em relação a um modelo totalmente ajustado:

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: -1.35%
rouge2: -1.70%
rougeL: -1.34%
rougeLsum: -1.35%


Aqui vemos uma pequena redução percentual nas métricas ROUGE em relação ao fully fine tune. No entanto, o treinamento requer muito menos recursos de computação e memória (geralmente apenas uma única GPU).